# Practice code for tool calling
## Task
- Allow parallel tool calling and sequential loops
- Record adding thorugh the model
- Function name matching efficiently
- Solve gradio's missing tool call history problem

""

In [ ]:
#environment setup
import os
import json
import gradio as gr
import random
import sqlite3
import requests
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

groq_endpoint = "https://api.groq.com/openai/v1/"
groq_key = os.getenv('GROQ_API_KEY')
MODEL2 = "moonshotai/kimi-k2-instruct-0905"
MODEL3 ="qwen/qwen3-32b"
MODEL = "gemini-3-flash-preview"

if not groq_key:
    print(f"\N{GHOST} Error: Key not found")
else:
    print(f"OK! Key found: {groq_key[:4]}... \n")

#testing whether the Groq endpoint is reachable
if groq_key:
    HEADERS = {
        "Authorization": f"Bearer {groq_key}",
        "Content-Type": "application/json"
    }
    ENDPOINT = "https://api.groq.com/openai/v1/models"

    try:
        response = requests.get(url=ENDPOINT, headers=HEADERS)
        print(response.json())
    except Exception as e:
        print(f"\N{GHOST} An error occured while testing groq's endpoint. Use gemini instead \n")
        repr(e)

gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta", api_key=os.getenv('GEMINI_API_KEY'))
    
groq = OpenAI(base_url=groq_endpoint, api_key=groq_key)

OK: Key found: gsk_...
{'object': 'list', 'data': [{'id': 'canopylabs/orpheus-arabic-saudi', 'object': 'model', 'created': 1765926439, 'owned_by': 'Canopy Labs', 'active': True, 'context_window': 4000, 'public_apps': None, 'max_completion_tokens': 50000}, {'id': 'whisper-large-v3', 'object': 'model', 'created': 1693721698, 'owned_by': 'OpenAI', 'active': True, 'context_window': 448, 'public_apps': None, 'max_completion_tokens': 448}, {'id': 'whisper-large-v3-turbo', 'object': 'model', 'created': 1728413088, 'owned_by': 'OpenAI', 'active': True, 'context_window': 448, 'public_apps': None, 'max_completion_tokens': 448}, {'id': 'llama-3.3-70b-versatile', 'object': 'model', 'created': 1733447754, 'owned_by': 'Meta', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 32768}, {'id': 'canopylabs/orpheus-v1-english', 'object': 'model', 'created': 1766186316, 'owned_by': 'Canopy Labs', 'active': True, 'context_window': 4000, 'public_apps': None, 'max_complet

In [2]:
#system message
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
Do not guess values, use the provided tools to get accurate values.
"""

In [3]:
#DB connection
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)")
    conn.commit()

In [ ]:
#get ticket price
def get_ticket_price(city, output=False):
    print(f"DATABASE TOOL CALLED. \n Getting ticket price for {city}...")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
    if output == True:
        return result
    else:
        return f"The ticket price for {city} is ${result[0]}" if result else f"No ticket price information for {city}. Do you wish to add this record?"


In [ ]:
#set ticket price
def set_ticket_price(city, price):
    print(f"DATABASE TOOL CALLED.\n Setting ticket price for {city} as ${price}...")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES(?,?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

    try:
        result = get_ticket_price(city, True)
        return f"The ticket price for {city} is ${result[0]}" if result else f"An error occured during this operation. Try again later"
    except Exception as e:
        repr(e)
        return f"An error occured during this operation. Try checking the ticket price for {city} again"


In [6]:
#add records
prices = []
cities = ["london", "new york", "nairobi", "new jersey", "paris", "alabama", "melborne", "arizona"]

for i in range(len(cities)):
    prices.append(random.randint(500, 2000))

for city, price in zip(cities, prices):
    set_ticket_price(city, price)

get_ticket_price("london")

DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for london...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for new york...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for nairobi...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for new jersey...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for paris...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for alabama...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for melborne...
DATABASE TOOL CALLED. Setting ticket price...
DATABASE TOOL CALLED. Getting ticket price for arizona...
DATABASE TOOL CALLED. Getting ticket price for london...


'The ticket price for london is $982.0'

In [7]:
#tool definitions

get_ticket_price_function = {
    "name": "get_ticket_price",
    "description": "Get the return ticket price for the city the user wants to go to",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the user wants to go to"
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    },
   
}

set_ticket_price_function = {
    "name": "set_ticket_price",
    "description": "Enter a record for a return ticket for a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type" : "string",
                "description": "The city that the user wants to add a record for"
            },
            "ticket_price": {
                "type": "integer",
                "description": "The price for the return ticket to the given city"
            },
        },
        "required": ["destination_city", "ticket_price"],
        "additionalProperties": False
    },
    
}

tools = [
    {
        "type": "function", 
        "function": get_ticket_price_function
    },
    {
        "type": "function",
        "function": set_ticket_price_function
    }
 ]

print(tools)

[{'type': 'function', 'function': {'name': 'get_ticket_price', 'description': 'Get the return ticket price for the city the user wants to go to', 'parameters': {'type': 'object', 'properties': {'destination_city': {'type': 'string', 'description': 'The city the user wants to go to'}}, 'required': ['destination_city'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'name': 'set_ticket_price', 'description': 'Enter a record for a return ticket for a given city', 'parameters': {'type': 'object', 'properties': {'destination_city': {'type': 'string', 'description': 'The city that the user wants to add a record for'}, 'ticket_price': {'type': 'integer', 'description': 'The price for the return ticket to the given city'}}, 'required': ['destination_city', 'ticket_price'], 'additionalProperties': False}}}]


## Using a function map to call listed tools only
This is made possible by creating a ```dictionary``` that ```maps``` a string to the corresponding ```function name```.

```NOTE```:
The functions are not called here ie., ```fn()```, they are just ```referenced``` to be used as a ```call back```.
This is also evident when passing functions while defining gradio interfaces


In [8]:
#handle tool calls and function mapping
function_map = {
    "get_ticket_price": {
        "function":get_ticket_price,
        "params": ["city"]
        },
    "set_ticket_price": {
        "function":set_ticket_price,
        "params": ["city", "price"]
        }
}

def handle_tool_calls(message):
    responses = []
    for tool in message.tool_calls:
        tool_function_name = tool.function.name
        arguments = json.loads(tool.function.arguments)
        if tool_function_name in function_map:
            if len(function_map[tool_function_name]["params"]) == 2:
                print(f"Context: city-{city}, price-{price}")
                result = function_map[tool_function_name]["function"](city=arguments.get("destination_city") , price=arguments.get("ticket_price"));
            else:
                result = function_map[tool_function_name]["function"](city=arguments.get("destination_city"));

            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool.id
            })
        return responses


## Solving the tool results issue 
I have made use of two lists here:
- The ```context``` list keeps track of the entire conversation
- The ```tool_results``` list keeps tract of the tool_calling responses

```History Manager```
This function takes in history from gradio UI, filters out what is already in the context to avoid duplication and
returns the context


In [9]:
#history manager
context = []
tool_results = []

def history_manager(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    index = len(context) - len(tool_results)
    context.extend(history[index:])

    return context
    

### Sequental tool calls through via multiple llm calls is made possible by using ```while```. 

In [10]:
#chat method
def chat(message, history):
    context = history_manager(history)
    user_message = [{"role": "user", "content": message}]
    messages=[{"role": "system", "content": system_message}] + context + user_message
    response = groq.chat.completions.create(model=MODEL3, messages=messages, tools= tools)
    context.extend(user_message)
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        print(message)
        results = handle_tool_calls(message)
        messages.append(message)
        messages.extend(results)
        context.extend(results)
        tool_results.extend(results)

        response = groq.chat.completions.create(model=MODEL3, messages=messages, tools=tools)
    print(context)
    return response.choices[0].message.content


In [ ]:
#gradio component

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'Hello Qwen'}]
[{'role': 'user', 'content': 'Hello Qwen'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'What roles do you play here?'}]
[{'role': 'user', 'content': 'Hello Qwen'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'What roles do you play here?'}, {'role': 'assistant', 'content': 'I assist with checking and updating return ticket prices for various cities.'}, {'role': 'user', 'content': 'Okay, so can I select any city?'}]
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='jatzr30rs', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')], reasoning='Okay, the user wants me to check the ticket price for London. If it\'s less than 1000, update it to 1000. If not, compare L

## NOTES ON TOOL CALLING LLMS
Some llms do not allow parallel tool calling and thus when used to make more than one tool call, you may encounter errors.

```Parallel tool calling``` allows models to determine that a user's single request requires more that one tool call and thus it populates its list of calls with the needed number of tool calls.

Some models that don't allow parallel tool calling:
- gpt-oss-120b
- gpt-oss-20b

Models that excell in parallel tool calling:
- qwen32b
- gemini-3-flash-preview

I've also noticed that the 2 models above handle their instructions very well and do not hallucinate by giving random values.

Contrary to this, ```moonshotai/kimi-k2-instruct-0905``` works well for the first couple tool calls but proceeds to provide random values instead of utilizing the tools.